In [4]:
import torch
import torch.nn as nn
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertModel, BertConfig
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup

import random
import time
import argparse
from utils import init_logger, load_tokenizer, read_prediction_text, set_seed, MODEL_CLASSES, MODEL_PATH_MAP, get_intent_labels, get_slot_labels, compute_metrics
import torch.nn.functional as F

import logging

logger = logging.getLogger()


from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

ModuleNotFoundError: No module named 'ray'

In [3]:
if __name__ == '__main__':
    time_wait = random.uniform(0, 10)
    time.sleep(time_wait)
    parser = argparse.ArgumentParser()

    parser.add_argument("--task", default='mixatis', type=str, help="The name of the task to train")

#     parser.add_argument("--model_dir", default='./gpsr_model', required=True, type=str, help="Path to save, load model")
    parser.add_argument("--model_dir", default='./mixatis_model', type=str, help="Path to save, load model")

    parser.add_argument("--data_dir", default="./data", type=str, help="The input data dir")
    parser.add_argument("--intent_label_file", default="intent_label.txt", type=str, help="Intent Label file")
    parser.add_argument("--slot_label_file", default="slot_label.txt", type=str, help="Slot Label file")
    parser.add_argument("--model_type", default="multibert", type=str, help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()))
#     parser.add_argument("--intent_seq", type=int, default=0, help="whether we use intent seq setting")
    parser.add_argument("--intent_seq", type=int, default=1, help="whether we use intent seq setting")

    parser.add_argument("--multi_intent", type=int, default=1, help="whether we use multi intent setting")
    parser.add_argument("--tag_intent", type=int, default=1, help="whether we can use tag to predict intent")

    parser.add_argument("--BI_tag", type=int, default=1, help='use BI sum or just B')
    parser.add_argument("--cls_token_cat", type=int, default=1, help='whether we cat the cls to the slot output of bert')
    parser.add_argument("--intent_attn", type=int, default=1, help='whether we use attention mechanism on the CLS intent output')
    parser.add_argument("--num_mask", type=int, default=7, help="assumptive number of slot in one sentence")
                                           #max slot num = 7



    parser.add_argument('--seed', type=int, default=25, help="random seed for initialization")
    parser.add_argument("--train_batch_size", default=128, type=int, help="Batch size for training.")
#     parser.add_argument("--train_batch_size", default=64, type=int, help="Batch size for training.")

    parser.add_argument("--eval_batch_size", default=256, type=int, help="Batch size for evaluation.")
    # gpsr
    # parser.add_argument("--max_seq_len", default=32, type=int, help="The maximum total input sequence length after tokenization.")

    # SNIPS ATIS
    # parser.add_argument("--max_seq_len", default=64, type=int, help="The maximum total input sequence length after tokenization.")#!!!!!!!!!!!! need toadd crop in unet!!!!
    parser.add_argument("--max_seq_len", default=50, type=int, help="The maximum total input sequence length after tokenization.")#!!!!!!!!!!!! need toadd crop in unet!!!!

    # parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")

#     parser.add_argument("--num_train_epochs", default=10.0, type=float, help="Total number of training epochs to perform.")
    parser.add_argument("--num_train_epochs", default=15.0, type=float, help="Total number of training epochs to perform.")
                                            #####

    parser.add_argument("--weight_decay", default=1e-4, type=float, help="Weight decay if we apply some.")




    parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1, type=float, help="Max gradient norm.")
    parser.add_argument("--max_steps", default=-1, type=int, help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
    parser.add_argument("--dropout_rate", default=0.1, type=float, help="Dropout for fully-connected layers")
    parser.add_argument('--logging_steps', type=int, default=500, help="Log every X updates steps.")
    parser.add_argument('--save_steps', type=int, default=300, help="Save checkpoint every X updates steps.")
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the test set.")
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument("--ignore_index", default=0, type=int,
                        help='Specifies a target value that is ignored and does not contribute to the input gradient')
    parser.add_argument('--slot_loss_coef', type=float, default=2.0, help='Coefficient for the slot loss.')
    parser.add_argument('--tag_intent_coef', type=float, default=1.0, help='Coefficient for the tag intent loss')

    # CRF option
    parser.add_argument("--use_crf", action="store_true", help="Whether to use CRF")
    parser.add_argument("--slot_pad_label", default="PAD", type=str, help="Pad token for slot label pad (to be ignore when calculate loss)")
    parser.add_argument("--patience", default=0, type=int, help="The initial learning rate for Adam.")

    parser.add_argument('-f')#########################

    args = parser.parse_args()

    args.model_name_or_path = MODEL_PATH_MAP[args.model_type]
    args.model_name_or_path = MODEL_PATH_MAP[args.model_type]

    tokenizer = load_tokenizer(args)

In [4]:
a = [1,2,3,4]
a[::-1]
b = [1,2,3]
b[::-1][0]

3

In [6]:
class UNet(nn.Module):
    def __init__(self, args,config, num_intent_labels, num_slot_labels,features = [800,900,1024]):
        super().__init__()
        self.embedding_dim = config.hidden_size
        self.max_seq_len = args.max_seq_len
        self.output_dim = num_intent_labels + num_slot_labels

        self.down_layers = nn.ModuleList([nn.Conv1d(in_channels = self.embedding_dim,out_channels = 800, kernel_size = 12,padding='same'),
                                         nn.Conv1d(in_channels = 800,out_channels = 900, kernel_size = 4,padding='same'),
                                         nn.Conv1d(in_channels = 900,out_channels = 1024, kernel_size = 3,padding='same')])

        self.bottle_neck = nn.Conv1d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding='same')


        self.up_layers = nn.ModuleList([nn.Conv1d(in_channels = 1024,out_channels = 900, kernel_size = 3, padding='same'),
                                         nn.Conv1d(in_channels = 900*2,out_channels = 800, kernel_size = 4,padding='same'),
                                         nn.Conv1d(in_channels = 800*2,out_channels = self.embedding_dim, kernel_size =12,padding='same')])

        self.conv_mask = nn.ModuleList([nn.Conv1d(in_channels = self.embedding_dim,out_channels = 256, kernel_size = 8,padding='same'),
                                         nn.Conv1d(in_channels = 256,out_channels = self.output_dim, kernel_size = 1)])

        self.pool = nn.MaxPool1d(2, return_indices=False)
        self.up_conv_trans = nn.ModuleList()
        #self.unpool = nn.Upsample(scale_factor=2, mode='nearest')

        lens = [self.max_seq_len]
        for i in range(len(features)):
            lens.append(lens[-1]//2) #floor, cuz max pool will floor

        conv_trans_ks = []
        cur_len = lens.pop()
        for l in lens[::-1]:
            conv_trans_ks.append(l - cur_len + 1)
            cur_len = l

        for i,ks in enumerate(conv_trans_ks):
            feat = features[::-1][i]
            self.up_conv_trans.append(nn.ConvTranspose1d(feat,feat, kernel_size=ks))


    def forward(self, embeded):
        x = embeded.permute(0,2,1)
        #print('permute size: ',embeded.size())
        skip_connections = []
        indices = []
        for i, layer in enumerate(self.down_layers):
            x = F.relu(layer(x))
            # print('x shape: ', x.size())

            if i < 2:
                skip_connections.append(x)
                x = self.pool(x)
                # print('small x size: ',x.size())
        # print('-------------------------------------------------')
        # bottle neck
        x = F.relu(self.bottle_neck(x))
        #print('-------------------------------------------------')
        for i, layer in enumerate(self.up_layers):
            if i > 0:
                # x = self.unpool(x)
                x = self.up_conv_trans[i](x)
                # print('large x size: ',x.size())
                sc = skip_connections.pop()
                #print('pop size: ', sc.size())
                #print('curr x size: ', x.size())
                x = torch.cat((sc,x),dim = 1)
                # print('!cat size: ',x.size())
            x = F.relu(layer(x))
            # if i < len(self.up_layers) - 1: x = F.relu(x)

        for i, layer in enumerate(self.conv_mask):
            x = layer(x)
            if i < len(self.conv_mask) - 1: x = F.relu(x)
        return (x)


config_class, model_class, _ = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.model_name_or_path, finetuning_task=args.task)
num_intent_labels = len(get_intent_labels(args))
num_slot_labels = len(get_slot_labels(args))
unet = UNet(args,config, num_intent_labels, num_slot_labels)
unet

UNet(
  (down_layers): ModuleList(
    (0): Conv1d(768, 800, kernel_size=(12,), stride=(1,), padding=same)
    (1): Conv1d(800, 900, kernel_size=(4,), stride=(1,), padding=same)
    (2): Conv1d(900, 1024, kernel_size=(3,), stride=(1,), padding=same)
  )
  (bottle_neck): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=same)
  (up_layers): ModuleList(
    (0): Conv1d(1024, 900, kernel_size=(3,), stride=(1,), padding=same)
    (1): Conv1d(1800, 800, kernel_size=(4,), stride=(1,), padding=same)
    (2): Conv1d(1600, 768, kernel_size=(12,), stride=(1,), padding=same)
  )
  (conv_mask): ModuleList(
    (0): Conv1d(768, 256, kernel_size=(8,), stride=(1,), padding=same)
    (1): Conv1d(256, 145, kernel_size=(1,), stride=(1,))
  )
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (up_conv_trans): ModuleList(
    (0): ConvTranspose1d(1024, 1024, kernel_size=(7,), stride=(1,))
    (1): ConvTranspose1d(900, 900, kernel_size=(14,), stride=(1,))
    (2)

In [10]:
config_class, model_class, _ = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.model_name_or_path, finetuning_task=args.task)

class JSF_Seg(nn.Module):
    def __init__(self, args,config, num_intent_labels, num_slot_labels, froze_last = True):
        super().__init__()

        self.bert = BertModel(config=config)
        self.unet = UNet(args,config, num_intent_labels, num_slot_labels)

        if froze_last:
            for name, param in self.bert.named_parameters():
                if name.startswith("encoder.layer.11"): # choose whatever you like here
                    param.requires_grad = True
                elif name.startswith("encoder.layer.10"):
                    param.requires_grad = True
                elif name.startswith("encoder.layer.9"):
                    param.requires_grad = True
                # elif name.startswith("encoder.layer.8"):
                #     param.requires_grad = True
                else:
                    param.requires_grad = False

    def forward(self,**inputs):
        outputs = self.bert(**inputs)
        sequence_output = outputs[0]
        out = self.unet(sequence_output)
        return out

num_intent_labels = len(get_intent_labels(args))
num_slot_labels = len(get_slot_labels(args))
model = JSF_Seg(args,config, num_intent_labels, num_slot_labels)
model

JSF_Seg(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

# The Training Loop

In [ ]:
from data_loader import load_and_cache_examples
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from torch.optim import AdamW
import numpy as np

class Trainer(object):
    def __init__(self, args, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset

        config_class, model_class, _ = MODEL_CLASSES[args.model_type]
        config = config_class.from_pretrained(args.model_name_or_path, finetuning_task=args.task)

        self.intent_dict = {i:label for i,label in enumerate(get_intent_labels(args))}
        self.slot_dict = {i:label for i,label in enumerate(get_slot_labels(args))}

        self.num_intent_labels = len(self.intent_dict)
        self.num_slot_labels = len(self.slot_dict)

        self.model = JSF_Seg(self.args,config, self.num_intent_labels, self.num_slot_labels)

        # GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
        self.model.to(self.device)

        # self.loss_function = nn.CrossEntropyLoss(ignore_index=self.args.ignore_index)
        self.loss_function = nn.CrossEntropyLoss()


    def get_losses(self, outputs,slot_label,intent_label):
        loss = 0
        for token_idx in range(outputs.shape[2]): # shape[2]: iterate through token level
            # true
            intent_true = intent_label[:,token_idx]
            slot_true = slot_label[:,token_idx]

            # preds
            intent_pred_one_hot = self.get_token_preds(outputs,token_idx, one_hot = True)['token_intent_pred']
            slot_pred_one_hot = self.get_token_preds(outputs,token_idx, one_hot = True)['token_slot_pred']

            # seperate losses
            intent_loss = self.loss_function(intent_pred_one_hot.float(),intent_true.long())
            slot_loss = self.loss_function(slot_pred_one_hot.float(),slot_true.long())

            # accumulate them to total loss
            loss += intent_loss + slot_loss

        return loss



    # get the token level prediction of the slot and
    def get_token_preds(self,outputs,token_idx, one_hot = True):
        token_pred = outputs[:,:,token_idx] #token_pred in shape: 64x17

        # get current token intention one-hot pred
        out_intent_pos = self.num_intent_labels
        token_intent_pred = token_pred[:,:out_intent_pos]

        start_slot_pos = self.num_intent_labels
        token_slot_pred = token_pred[:,start_slot_pos:]

        if one_hot:
            # size of BATCH_SIZE * MAX_SEN_LEN
            return {'token_intent_pred':token_intent_pred, 'token_slot_pred':token_slot_pred}

        else:
            prediction_intent = torch.argmax(token_intent_pred, dim = 1)
            prediction_slot = torch.argmax(token_slot_pred, dim = 1)
            return {'token_intent_pred':prediction_intent, 'token_slot_pred':prediction_slot}


    # get the setence level prediction of all the slots and intents
    # output: dict of sentence slot and intent labels lst
    def get_sentence_preds(self, outputs):
        # print(outputs[:,:self.num_intent_labels,:].shape)
        # print(outputs[:,self.num_intent_labels:,:].shape)
        batch_intent_pred = torch.argmax(outputs[:,:self.num_intent_labels,:], dim = 1).detach().cpu().tolist()# size 64 * 32
        start_slot_pos = self.num_intent_labels

        # print(f'outputs[:,start_slot_pos:,:][0]: {outputs[:,start_slot_pos:,:][0]}')
        batch_slot_pred = torch.argmax(outputs[:,start_slot_pos:,:], dim = 1).detach().cpu().tolist()# size 64 * 32
        # print(f'batch_slot_pred[0]:  {batch_slot_pred[0]}')
        # print(f'batch_slot_pred shape: {len(batch_slot_pred)} x {len(batch_slot_pred[0])}')


        intent_pred_labels = [[self.intent_dict[id] for id in sentence] for sentence in batch_intent_pred]
        slot_pred_labels = [[self.slot_dict[id] for id in sentence] for sentence in batch_slot_pred]

        return {'slot_sentence_pred':slot_pred_labels, 'intent_sentence_pred':intent_pred_labels}


    def train(self):
        train_sampler = RandomSampler(self.train_dataset)
        train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size)

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': self.args.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total)

        # Train!
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(self.train_dataset))
        logger.info("  Num Epochs = %d", self.args.num_train_epochs)
        logger.info("  Total train batch size = %d", self.args.train_batch_size)
        logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
        logger.info("  Total optimization steps = %d", t_total)
        logger.info("  Logging steps = %d", self.args.logging_steps)
        logger.info("  Save steps = %d", self.args.save_steps)


        global_step = 0
        tr_loss = 0.0
        self.model.zero_grad()

        train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")

        for n_ep,_ in enumerate(train_iterator):
            ep_loss = 0.0
            ep_step = 0
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                ep_step += 1
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU

                inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids' : batch[2]}

                outputs = self.model(**inputs) #outpus size: [64, 17, 32]
                ################################ define loss ################################
                #could do a token-level CE loss or a sentence mask level IoU or DIce score or fiscal

                #intent loss
                loss = 0

                slot_label = batch[4]
                intent_label = batch[5]

                # accumulate current loss to total loss
                loss += self.get_losses(outputs,slot_label,intent_label)

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()
                tr_loss += loss.item()
                ep_loss += loss.item()

                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1


                    # if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                    #     continue
                    #     self.evaluate("dev")

                    # if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                    #     self.save_model()

                if 0 < self.args.max_steps < global_step:
                    epoch_iterator.close()
                    break

            if 0 < self.args.max_steps < global_step:
                train_iterator.close()
                break

            print(f'training epoch: {n_ep + 1} current training loss: {ep_loss / ep_step}')
            print('---------------------- eval on dev -----------------------------')
            print(self.evaluate("dev"))
        return global_step, tr_loss / global_step

    ######################################################################## EVAL ########################################################################
    def evaluate(self, mode):
        if mode == 'test':
            dataset = self.test_dataset
        elif mode == 'dev':
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation on %s dataset *****", mode)
        logger.info("  Num examples = %d", len(dataset))
        logger.info("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0

        #self.model.eval()

        # init pred/true lists
        slot_sentence_pred = []
        intent_sentence_pred = []
        true_slot_label = []
        true_intent_label  = []

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids' : batch[2]}
                outputs = self.model(**inputs)

                slot_label = batch[4]
                intent_label = batch[5]
                eval_loss += self.get_losses(outputs,slot_label,intent_label)

                #print pred labels
                sentence_pred_dict = self.get_sentence_preds(outputs)

                slot_sentence_pred.extend(sentence_pred_dict['slot_sentence_pred'])
                intent_sentence_pred.extend(sentence_pred_dict['intent_sentence_pred'])

                true_slot_label.extend( [[self.slot_dict[id] for id in sentence]for sentence in slot_label.tolist()])
                true_intent_label.extend( [[self.intent_dict[id] for id in sentence]for sentence in intent_label.tolist()])

                # print(f'true_slot_label: {true_slot_label[0]}')
                # print(f'slot_sentence_pred: {slot_sentence_pred[0]}\n')
                #
                # print(f'true_intent_label: {true_intent_label[0]}')
                # print(f'intent_sentence_pred: {intent_sentence_pred[0]}\n')

            nb_eval_steps += 1

        eval_loss = eval_loss.item() / nb_eval_steps
        results = {
            "loss": eval_loss
        }

        #string labels
        slot_preds_list = slot_sentence_pred
        out_slot_label_list = true_slot_label

        intent_preds_list = intent_sentence_pred
        out_intent_label_list = true_intent_label

        # compute metrics
        the_results = compute_metrics(intent_preds_list, out_intent_label_list,slot_preds_list, out_slot_label_list)

        results.update(the_results)

        logger.info("***** Eval results *****")
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))

        return results

train_dataset = load_and_cache_examples(args, tokenizer, mode="train")
dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev")
test_dataset = load_and_cache_examples(args, tokenizer, mode="test")
trainer = Trainer(args,  train_dataset, dev_dataset,  test_dataset)
trainer.train()

Iteration:   0%|          | 0/141 [00:00<?, ?it/s]/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,

Iteration: 100%|██████████| 141/141 [00:57<00:00,  2.46it/s]


training epoch: 1 current training loss: 86.02952064351832
---------------------- eval on dev -----------------------------



Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.36it/s]
/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: UNK seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: atis_capacity seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: atis_flight seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/crossing/miniconda3/envs/jointbert/lib/python3.9/site-packages/seqeva

{'loss': 40.586341857910156, 'intent_precision': 0.40663436451733137, 'intent_recall': 0.34822853495052664, 'intent_f1': 0.375171939477304, 'slot_precision': 0.6099808061420345, 'slot_recall': 0.47221396731054976, 'slot_f1': 0.5323283082077052, 'sementic_frame_acc': 0.004}



Iteration: 100%|██████████| 141/141 [00:57<00:00,  2.46it/s]


...


Iteration: 100%|██████████| 141/141 [00:58<00:00,  2.41it/s]


training epoch: 7 current training loss: 2.5793861573469554
---------------------- eval on dev -----------------------------



Epoch:  47%|████▋     | 7/15 [07:03<08:06, 60.75s/it]

{'loss': 13.053539276123047, 'intent_precision': 0.7946875, 'intent_recall': 0.8116820938397702, 'intent_f1': 0.8030948997315649, 'slot_precision': 0.8066968855095774, 'slot_recall': 0.8197622585438336, 'slot_f1': 0.8131770948485518, 'sementic_frame_acc': 0.212}



Iteration:  60%|█████▉    | 84/141 [00:34<00:23,  2.40it/s]

In [ ]:
trainer.evaluate('test')